In [271]:
from ttrtypes import Type,BType,PType,Pred,Possibility,RecType,Fun,ListType,\
                     TTRStringType,KPlusStringType,LazyObj,ti_apply,SingletonType,FunType,JoinType
from records import Rec
from utils import show

# Very basic update

In [346]:
Ind = BType('Ind')
Ind.judge('A')
Ind.judge('B')
greet = Pred('Greet',[Ind,Ind])
countergreet = Pred('CounterGreet',[Ind,Ind])

Greet = lambda x, y : PType(greet, [x,y])
Greet = Fun('x', Ind, Fun('y', Ind, PType(greet, ['x','y'])))
Greet = RecType({'x':Ind, 'y':Ind, 'e':(Fun('v1', Ind, Fun('v2', Ind, PType(greet, ['v1', 'v2']))), ['x','y'])})

#show(Greet.appc('A').appc('B'))

IllocRel = Type('IllocRel')
IllocRel.judge(Greet)

IllocProp = RecType({"R": IllocRel,
                               "x": Ind,
                               "y": Ind,
                               "sit-type": (Fun('v1',Ind,
                                     Fun('v2',Ind, 
                                         Fun('v3',IllocRel,
                                             RecType({'c1': LazyObj([LazyObj(['v3','@','v1'])
                                                                     ,'@','v2'])})))),
                                  ['R','x','y']
                                 )})

illocPropRec = Rec({"R": Greet, "x":"A", "y":"B", "sit-type": Greet})

#show(IllocProp.query(illocPropRec))
print(show(illocPropRec))
print(show(IllocProp))
M = Possibility('M')
show(IllocProp.in_poss(M).query(illocPropRec))

{R = {x : Ind, y : Ind, e : (lambda v1:Ind . lambda v2:Ind . Greet(v1, v2), [x, y])}, x = A, y = B, sit-type = {x : Ind, y : Ind, e : (lambda v1:Ind . lambda v2:Ind . Greet(v1, v2), [x, y])}}
{R : IllocRel, x : Ind, y : Ind, sit-type : (lambda v1:Ind . lambda v2:Ind . lambda v3:IllocRel . {c1 : [[v3, @, v1], @, v2]}, [R, x, y])}


AttributeError: 'NoneType' object has no attribute 'appc_m'

In [ ]:


IllocProp = lambda r: RecType({"R": IllocRel,
                     "x": Ind,
                     "y": Ind,
                     "sit-type": (Fun('v1',Ind,
                                     Fun('v2',Ind, 
                                         PType(r, ['v1','v2']))))
                    })



In [327]:
show(Fun('v1',Ind,
    Fun('v2',Ind, 
        Fun('v3',IllocRel,
            RecType({'c1': LazyObj([LazyObj(['v3','@','v1'])
                                    ,'@','v2'])})))).app("A").app("B").app(Greet))

'{c1 : Greet(A, B)}'

In [328]:
IllocProp = RecType({"R": IllocRel,
                     "x": Ind,
                     "y": Ind,})
illocPropRec = Rec({"R": Greet, "x":"A", "y":"B"})
show(IllocProp.query(illocPropRec))

'True'

In [307]:
PreCond = RecType({'spkr':Ind,
                   'addr':Ind,
                   'moves': SingletonType(ListType(IllocProp), [])})

In [309]:
greeting = Fun('r', 
               PreCond, 
               RecType({'moves': SingletonType(ListType(IllocProp),
                                               [Greet.app(LazyObj(['r','.','spkr'])).app(LazyObj(['r','.','addr']))])}))
show(greeting)

'lambda r:{spkr : Ind, addr : Ind, moves : [{R : IllocRel}]_[]} . {moves : [{R : IllocRel}]_[Greet([r, ., spkr], [r, ., addr])]}'

In [281]:
greeting = Fun('r', 
               PreCond, 
               RecType({'moves': SingletonType(ListType(IllocProp),
                                               ListType(PType(greet,[LazyObj(['r','.','spkr']),
                                                                     LazyObj(['r','.','addr'])])))}))
show(greeting)

'lambda r:{spkr : Ind, addr : Ind, moves : [T8]_[]} . {moves : [T8]_[Greet([r, ., spkr], [r, ., addr])]}'

In [282]:
no_turn_change = Fun('r', 
                    RecType({'spkr': Ind, 'addr': Ind}),
                    RecType({'spkr': SingletonType(Ind, LazyObj(['r','.','spkr'])),
                             'addr': SingletonType(Ind, LazyObj(['r','.','addr']))}))
turn_change =    Fun('r', 
                    RecType({'spkr': Ind, 'addr': Ind}),
                    RecType({'spkr': SingletonType(Ind, LazyObj(['r','.','addr'])),
                             'addr': SingletonType(Ind, LazyObj(['r','.','spkr']))}))

In [284]:
Ind.judge('A')
Ind.judge('B')

#IS = Rec({'spkr':'A', 'addr':'B', 'moves':[]})
DGBType = RecType({'spkr':SingletonType(Ind, 'A'), 
                    'addr':SingletonType(Ind, 'B'), 
                    'moves': SingletonType(ListType(IllocProp), [])})
IS = DGBType.create()
print("initial IS     :", show(IS))
print("DGB type for it:", show(DGBType))

initial IS     : {spkr = A, addr = B, moves = []}
DGB type for it: {spkr : Ind_A, addr : Ind_B, moves : [T8]_[]}


In [277]:
print(show(DGBType), show(IS))
DGBType = DGBType.amerge(turn_change.appc(IS))
IS = DGBType.create()
print(show(DGBType), show(IS))
DGBType = DGBType.amerge(greeting.appc(IS))
IS = DGBType.create()
print(show(DGBType), show(IS))

{spkr : Ind_A, addr : Ind_B, moves : [T8]_[]} {spkr = A, addr = B, moves = []}
{spkr : Ind_B, addr : Ind_A, moves : [T8]_[]} {spkr = B, addr = A, moves = []}
{moves : [T8]_[Greet(B, A)], spkr : Ind_B, addr : Ind_A} {moves = [Greet(B, A)], spkr = B, addr = A}


'{spkr = _a46, addr = _a47, moves = [Greet(A, B)]}'